In [9]:
############################################################
# 
# Synchronisation de la copie de HAL : EMBARGOS
# Téléchargement des fichiers qui ne sont plus sous embargo
#
############################################################
import pandas as pd
from datetime import datetime
import os
import requests
import logging
import time



# Obtenir la date actuelle
date_extraction_current = datetime.now().strftime("%Y-%m-%d")

# Temps de départ
start_time = time.time()

# Durée d'exécution en secondes (1 minute = 60 secondes)
duration = 10000  # en secondes


## Spécifier le répertoire de log
log_directory = '/data/log/'
## Créer le répertoire s'il n'existe pas
os.makedirs(log_directory, exist_ok=True)

# Construire le nom du fichier de log avec le chemin complet
log_file = os.path.join(log_directory, date_extraction_current + '_log_synchro_embargo_hal.txt')

# Configuration du logger
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s', filemode='w')


# Charger le fichier Embargos.xlsx
embargos_file_path = "/data/log/Embargos.xlsx"
df = pd.read_excel(embargos_file_path)

logging.info(f"nombre de lignes fichier embargos : {len(df)}")

# Récupérer la date actuelle
today_date = datetime.today().date()

# Convertir la colonne ref_date en date et ajouter une nouvelle colonne temporaire
df['ref_date_converted'] = pd.to_datetime(df['ref_date'], errors='coerce').dt.date

# Comparer la colonne 'ref_date_converted' avec la date d'aujourd'hui
df['télécharger'] = df['ref_date_converted'].apply(lambda x: 'Oui' if pd.notna(x) and x < today_date else '')

# variable pour restreindre les extensions à télécharger
include_file_extension=("pdf", "PDF", "doc", "DOC","docx","DOCX","odt","ODT","ppt","PPT","pptx","PPTX","rtf","RTF")


# Dossier pour enregistrer les fichiers téléchargés
download_folder = "/data/pdf_de_hal"
os.makedirs(download_folder, exist_ok=True)

# Télécharger les fichiers si nécessaire
for index, row in df.iterrows():
    #print(row['télécharger'])
    #print(hal_id)

    if time.time() - start_time > duration:
            print("Temps écoulé, arrêt de l'exécution.")
            break

    if row['télécharger'] == 'Oui':
        # Télécharger le fichier ici 
        target_url = row['target']

        if not target_url.endswith(include_file_extension):
            # print(f"pas la bonne extension :", target_url)
            continue
        # print (target_url)    

        #print(f"url : {target_url}")
        hal_id = row['halID_value']
        main = row['embargomainfile']
        file_main = os.path.basename(target_url)  
        name, ext = os.path.splitext(file_main)   
        #print(f"Nom : {name} - Ext : {ext}")

        # Réduire le nom à 100 caractères en incluant l'extension
        max_length = 100
        if len(file_main) > max_length:
            truncated_name = name[:max_length - len(ext)]  # Calcul de la longueur restante pour le nom
            file_main = f"{truncated_name}{ext}"          # Concaténation avec l'extension
            

        if main == True:
            file_main = f"main_{file_main}"

        file_name = os.path.join(download_folder, f"{hal_id}_{file_main}")
                   
        # print(file_name)  

        try:
            response = requests.get(target_url)
            response.raise_for_status()  # Vérifie si la requête a échoué
            with open(file_name, 'wb') as file:
                file.write(response.content)
            #logging.info(f"Téléchargé : {file_name}")
            # print (f"Téléchargé : {file_name}")
        except requests.exceptions.RequestException as e:
            logging.info(f"Erreur lors du téléchargement de {target_url}: {e}")
    # else:
        #print("Pas de date antérieure à aujourd'hui")

# Supprimer les lignes où 'téléchargé' est 'Oui'
df = df[df['télécharger'] != 'Oui']

# Supprimer la colonne temporaire 'ref_date_converted'
df = df.drop(columns=['ref_date_converted'])

# Enregistrer le fichier Excel modifié (sans les lignes téléchargées et sans la colonne temporaire)
df.to_excel(embargos_file_path, index=False)

print(f"Processus Embargos terminé.")
logging.info(f"Processus Embargos terminé")

# Appeler shutdown pour fermer les handlers du fichier de log
logging.shutdown()


Processus Embargos terminé.
